In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import os
import unicodedata
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [45]:
dat = pd.read_csv('tidy_data.csv')
gwb = pd.read_csv('tidy_data_gwb.csv')
oba = dat.query('source == "oba"')
gwb2 = gwb.drop(['date', 'source', 'enc_value'], axis=1)
oba2 = oba.drop(['date', 'source', 'enc_value'], axis=1)
gwb = gwb.drop(['enc_value'], axis=1)
oba = oba.drop(['enc_value'], axis=1)
columnsTitles = list(oba.columns)
gwb = gwb.reindex(columns=columnsTitles)

<A HREF="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas cheat sheet</A>

In [46]:
oba.describe()

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,...,poly_word_count,dc_word_perc,gf_word_perc,poly_word_perc,ari,flesch_kincaid,coleman_liau,dale_chall,gunning_fog,smog
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.067531,0.068038,0.067378,0.067097,0.067643,0.068023,0.032871,0.068041,0.058334,0.066472,...,342.600000,0.325630,0.117772,0.132699,9.796724,10.011069,9.013232,15.216733,12.835918,12.456703
std,0.004032,0.004444,0.003886,0.004267,0.004120,0.004374,0.025976,0.004533,0.010906,0.004107,...,256.737692,0.031002,0.018802,0.022420,2.226838,1.842675,1.194147,1.866037,1.732366,1.200682
min,0.058444,0.060121,0.059642,0.053903,0.059658,0.060021,0.000000,0.059866,0.000000,0.057288,...,21.000000,0.249196,0.079503,0.089823,5.572010,6.538509,6.452430,11.745237,9.474320,9.994967
25%,0.064596,0.064551,0.064423,0.064082,0.064349,0.064822,0.000000,0.064618,0.056226,0.063562,...,162.000000,0.305598,0.105752,0.118932,8.607661,8.911693,8.223367,14.277198,11.709798,11.607377
50%,0.067778,0.068167,0.067654,0.067164,0.067797,0.068060,0.038543,0.068103,0.060636,0.066798,...,253.500000,0.325756,0.117567,0.129999,9.682680,9.976950,9.029858,15.188358,12.776164,12.452591
75%,0.070712,0.071145,0.070544,0.070578,0.070867,0.071120,0.057884,0.071156,0.064089,0.069083,...,485.250000,0.342618,0.130029,0.146754,10.577183,10.985449,9.739209,15.887708,13.692420,13.116943
max,0.075469,0.084538,0.075100,0.076752,0.078377,0.083426,0.073501,0.085095,0.078067,0.081758,...,1106.000000,0.394818,0.176545,0.199855,23.370658,20.722070,12.649953,27.950188,23.152281,18.243606


In [47]:
gwb.describe()

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,...,poly_word_count,dc_word_perc,gf_word_perc,poly_word_perc,ari,flesch_kincaid,coleman_liau,dale_chall,gunning_fog,smog
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,...,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,0.072602,0.073035,0.067219,0.069708,0.071687,0.073332,0.009207,0.074041,0.051332,0.068334,...,220.146067,0.369742,0.156861,0.175238,11.296091,11.677720,10.650114,16.075418,14.530251,13.892367
std,0.005475,0.005492,0.012966,0.007237,0.005900,0.005921,0.019494,0.006785,0.024217,0.009795,...,228.024348,0.055415,0.040353,0.043496,3.875644,3.143173,1.966051,3.532438,3.422347,2.371306
min,0.058520,0.061513,0.003105,0.035637,0.047651,0.061840,0.000000,0.061949,0.000000,0.000000,...,10.000000,0.262793,0.083695,0.091500,4.920833,6.643537,6.805726,10.437930,9.681578,10.117482
25%,0.069122,0.069369,0.065988,0.067114,0.068018,0.069533,0.000000,0.069474,0.045395,0.066538,...,54.000000,0.335619,0.132394,0.148607,8.898751,9.704546,9.434790,13.923981,12.336842,12.340627
50%,0.072379,0.072321,0.070409,0.069864,0.072150,0.072371,0.000000,0.072704,0.060778,0.069643,...,138.000000,0.367347,0.144404,0.164948,10.309189,10.909611,10.287196,15.129664,13.291594,13.216016
75%,0.075669,0.075843,0.072331,0.072395,0.074313,0.076392,0.000631,0.076561,0.068011,0.072640,...,283.000000,0.393241,0.176010,0.196013,12.840667,13.193030,11.594909,17.288542,15.500717,14.424011
max,0.091029,0.090538,0.088203,0.093270,0.090689,0.094317,0.066699,0.096735,0.083895,0.084116,...,913.000000,0.547059,0.311765,0.340441,28.814339,25.464086,17.659618,33.990996,28.514978,21.606579


## Obama vs GWB PCA

In [48]:
combined = pd.concat([oba,gwb], axis=0)
pca_data = combined.drop(['date', 'source'], axis=1)

In [49]:
# Scale data before applying PCA
scaling = StandardScaler()
 
# Use fit and transform method
scaling.fit(pca_data)
scaled_data = scaling.transform(pca_data)

pca = PCA(n_components=2)
pcafit = pca.fit_transform(scaled_data)
 
# Check the dimensions of data after PCA
print(pcafit.shape)

(189, 2)


In [50]:
# check how much variance is explained by each principal component
print(pca.explained_variance_ratio_)

[0.34970681 0.15580095]


In [51]:
fig = px.scatter(pcafit, 
                 x=0, 
                 y=1, 
                 color=combined['source'],
                 hover_name=combined['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

In [44]:
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=pca_data.columns)
loadings.sort_values(by=['PC1'], inplace=True)
loadings

,PC1,PC2
num_sents,-0.195848,0.127010
sent_count,-0.195843,0.126996
num_unique_words,-0.192640,0.142311
word_count,-0.190147,0.149299
num_words,-0.189581,0.149768
char_count,-0.185659,0.158219
syl_count,-0.184780,0.159498
dc_word_count,-0.181581,0.166724
gf_word_count,-0.167934,0.182111
poly_word_count,-0.167702,0.182624


## Obama PCA

In [53]:
# Scale data before applying PCA
scaling_oba = StandardScaler()
 
# Use fit and transform method
scaling_oba.fit(oba2)
scaled_data_oba = scaling_oba.transform(oba2)

pca_oba = PCA(n_components=2)
pcafit_oba = pca_oba.fit_transform(scaled_data_oba)
 
# Check the dimensions of data after PCA
print(pcafit_oba.shape)

(100, 2)


In [54]:
# check how much variance is explained by each principal component
print(pca_oba.explained_variance_ratio_)

[0.36426631 0.19864138]


In [57]:
fig_oba = px.scatter(pcafit_oba, 
                 x=0, 
                 y=1, 
                 color=oba['source'],
                 hover_name=oba['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig_oba.show()

In [58]:
loadings_oba = pd.DataFrame(pca_oba.components_.T, columns=['PC1', 'PC2'], index=oba2.columns)
loadings_oba.sort_values(by=['PC1'], inplace=True)
loadings_oba

,PC1,PC2
num_sents,-0.167943,0.154004
sent_count,-0.167927,0.153984
INTJ,-0.161868,-0.013985
word_count,-0.155155,0.181418
num_words,-0.154263,0.182002
num_unique_words,-0.153521,0.180301
SYM,-0.149424,0.052812
char_count,-0.148192,0.192360
syl_count,-0.146850,0.194610
dc_word_count,-0.145912,0.198768
